In [4]:
from keras.preprocessing.image import ImageDataGenerator

# Directories for your train and validation datasets
train_dir = (r'C:\Users\shaur\OneDrive\Desktop\archive (2\archive (2)\train')
validation_dir = (r'C:\Users\shaur\OneDrive\Desktop\archive (2\archive (2)\test')

# Dimensions to which to resize input images
image_size = (48, 48)
# image_size = (224, 224)# for ggogle net model
# Batch size
batch_size = 48

# # Rescale pixel values from [0, 255] to [0, 1]
# datagen = ImageDataGenerator(rescale=1./255)

# # Generator for the training data
# train_generator = datagen.flow_from_directory(
#     train_dir,
#     target_size=image_size,
#     batch_size=batch_size,
#     color_mode='grayscale',  # because the FER2013 images are grayscale
#     class_mode='categorical'  # because we have multiple classes (emotions)
# )

# Data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,       # rescale pixel values from [0, 255] to [0, 1]
    rotation_range=20,    # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range=0.2,       # randomly zoom image 
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
)

# Generator for the training data
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical'
)

# Generator for the validation data
validation_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=image_size,
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical'
)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\shaur\\OneDrive\\Desktop\\archive (2\\archive (2)\\train'

In [5]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import Adam

# Define the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))

model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(7, activation='softmax'))  # 7 is the number of classes (emotions)

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,  # for example
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)
model.save('emotion_model.h5')

NameError: name 'train_generator' is not defined

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np

# Define the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))  # Dropout layer after first pooling layer

# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Dropout(0.25))  # Dropout layer after second pooling layer

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))  # Dropout layer after third pooling layer

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))  # Dropout layer before final layer

model.add(Dense(7, activation='softmax'))  # 7 is the number of classes (emotions)

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=50,  # for example
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

model.save('emotion_model.h5')

# Number of classes
num_classes = 7

# Predict the classes on the validation dataset
Y_pred = model.predict(validation_generator, len(validation_generator))
y_pred = np.argmax(Y_pred, axis=1)

# Get the true class labels
y_true = validation_generator.classes

# Create a confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Display the confusion matrix
cmd = ConfusionMatrixDisplay(cm, display_labels=range(num_classes))
cmd.plot()

# Plot the training and validation accuracy
plt.figure()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()


Epoch 1/50
599/599 [==============================] - 733s 1s/step - loss: 1.8208 - accuracy: 0.2447 - val_loss: 1.7912 - val_accuracy: 0.2458
Epoch 2/50
311/599 [==============>...............] - ETA: 18s - loss: 1.7952 - accuracy: 0.2568

In [2]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout,BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import Callback, ModelCheckpoint
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
import matplotlib.pyplot as plt
import numpy as np

# Custom callback for accuracy comparison after each epoch
class AccuracyComparisonCallback(Callback):
    def __init__(self, validation_generator):
        super(AccuracyComparisonCallback, self).__init__()
        self.validation_generator = validation_generator

    def on_epoch_end(self, epoch, logs=None):
        Y_pred = self.model.predict(self.validation_generator, len(self.validation_generator))
        y_pred = np.argmax(Y_pred, axis=1)
        y_true = self.validation_generator.classes

        accuracy = accuracy_score(y_true, y_pred)

        print(f'Validation accuracy: {logs["val_accuracy"]}, Predicted accuracy: {accuracy}')

# # Define the model
# model = Sequential()
# model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
# model.add(MaxPooling2D((2, 2)))
# model.add(Dropout(0.25))  # Dropout layer after first pooling layer
# 
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Dropout(0.25))  # Dropout layer after second pooling layer
# 
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(Flatten())
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.5))  # Dropout layer before final layer
# 
# model.add(Dense(7, activation='softmax'))  # 7 is the number of classes (emotions)

# Define the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(BatchNormalization())  # Batch normalization layer
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))  # Dropout layer after first pooling layer

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())  # Batch normalization layer
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))  # Dropout layer after second pooling layer

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())  # Batch normalization layer
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))  # Dropout layer before final layer

model.add(Dense(7, activation='softmax'))  # 7 is the number of classes (emotions)


# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Specify the checkpoint file and define the ModelCheckpoint callback
checkpoint_filepath = 'model.{epoch:02d}-{val_loss:.2f}.h5'
model_checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath, save_best_only=False)

accuracy_comparison_callback = AccuracyComparisonCallback(validation_generator)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=50,  # for example
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=[model_checkpoint_callback, accuracy_comparison_callback]
)

model.save('emotion_model.h5')

# Number of classes
num_classes = 7

# Predict the classes on the validation dataset
Y_pred = model.predict(validation_generator, len(validation_generator))
y_pred = np.argmax(Y_pred, axis=1)

# Get the true class labels
y_true = validation_generator.classes

# Create a confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Display the confusion matrix
cmd = ConfusionMatrixDisplay(cm, display_labels=range(num_classes))
cmd.plot()

# Plot the training and validation accuracy
plt.figure()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()


NameError: name 'validation_generator' is not defined

In [ ]:
from mtcnn import MTCNN
import cv2
import numpy as np
from keras.models import load_model

# Load pre-trained emotion classification model
emotion_model = load_model('emotion_model.h5')

# Emotion labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Load the image
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    if not ret:
        print("Failed to grab frame")
        break

    cv2.imshow("Press 'q' or 'Q' to take a picture", frame)

    key = cv2.waitKey(1)
    if key & 0xFF == ord('q') or key & 0xFF == ord('Q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

detector = MTCNN()

# Convert to grayscale for face detection
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# Perform face detection
faces = detector.detect_faces(frame)

# Loop over all detected faces
for person in faces:
    bounding_box = person['box']

    x, y, w, h = bounding_box

    # Extract face
    face = gray[y:y+h, x:x+w]

    # Resize and normalize the face image to the size your emotion model expects
    face = cv2.resize(face, (48, 48))
    face = face / 255.0

    # The model expects a 4D tensor, so expand dimensions
    face = np.expand_dims(face, axis=(0, -1))

    # Predict emotion
    emotion = emotion_model.predict(face)

    # Get the emotion label
    emotion_label = emotion_labels[np.argmax(emotion)]

    # Print the emotion label
    print(emotion_label)

    # Draw the bounding box of the face along with the emotion label
    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
    cv2.putText(frame, emotion_label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

# Show the image
cv2.imshow('Image', frame)
cv2.waitKey(0)
cv2.destroyAllWindows()
